# Loan Default Prediction System - Model Training

This notebook is designed to run the training pipeline with full hyperparameter tuning (XGBoost, CatBoost, etc.) on Google Colab's GPU.

In [ ]:
# 1. Clone the Repository
!git clone https://github.com/rkpcode/loan_defult_prediction_system.git
%cd loan_defult_prediction_system

In [ ]:
# 2. Install Dependencies
!pip install -r requirements.txt
!pip install -e .  # Now works because setup.py is fixed

In [ ]:
# 3. Fix Python Path (Crucial for 'src' imports)
import sys
import os

# Add the current directory to sys.path so 'src' module can be found
sys.path.append(os.getcwd())
print(f"Current working directory: {os.getcwd()}")
print("Updated sys.path to include current directory.")

In [ ]:
# 4. Run Training Pipeline
!python src/loan_defult_prediction_system/pipelines/training_pipeline.py

In [ ]:
# 5. Detailed Evaluation & Model Metadata
import json
if os.path.exists('artifacts/model_metadata.json'):
    with open('artifacts/model_metadata.json', 'r') as f:
        print(json.load(f))
else:
    print("Model metadata not found. Training might have failed.")

In [ ]:
# 6. Download the Best Model
from google.colab import files
if os.path.exists('artifacts/model.pkl'):
    files.download('artifacts/model.pkl')
if os.path.exists('artifacts/model_metadata.json'):
    files.download('artifacts/model_metadata.json')